### script to tag the articles and store the articles in a collection in 

In [1]:
from pymongo import MongoClient

In [2]:
from bson.objectid import ObjectId

In [3]:
client = MongoClient(host='act4dgem.cse.iitd.ac.in')

In [4]:
local_client = MongoClient()

In [5]:
local_client.database_names()

['admin', 'config', 'ikb', 'local', 'media-db']

In [6]:
tagged_articles_coll = local_client.get_database('media-db').get_collection('taggedArticles')

In [9]:
tagged_articles_coll.delete_many({})

In [14]:
local_client.get_database('media-db').list_collection_names()

['test', 'taggedArticles']

In [15]:
mediaDb = client.get_database('media-db')

In [16]:
resolvedEntityDb = client.get_database('eventwise_media-db')

In [17]:
articles = mediaDb.get_collection('articles')

In [18]:
resolved_entities = resolvedEntityDb.get_collection('all_media_entities_resolved_final')

In [19]:
def updateText(objectId, text, entype, offset, length):
    if entype == 'Person':
        otag = '<PER>'
        ctag = '</PER>'
    elif entype == 'Company' or entype == 'Organization':
        otag = '<ORG>'
        ctag = '</ORG>'
    #elif entype == 'Country' or entype == 'City' or entype =='ProvinceOrState' or entype =='Continent':
    elif entype == 'City':
        otag = '<LOC>'
        ctag = '</LOC>'

    elif entype == 'ProvinceOrState':
        otag = '<STA>'
        ctag = '</STA>'
        
    else :
        otag = '<MISC>'
        ctag = '</MISC>'
    p1 = text[:offset]
    p2 = otag + str(objectId) + ctag
    p3 = text[offset+length:]

    shift = len(p2) - length
    return (p1 + p2 + p3, shift)

In [20]:
def updateOffset(entities, old_offset, shift):
    #print 'Updated Offset'
    for e in entities:
        for inst in e['instances']:
            #print inst
            if inst['offset'] > old_offset:
                inst['offset']+=shift
    return entities


In [25]:
def copyText():
    global tagged_articles_coll
    for a in tagged_articles_coll.find({}):
        try:
            tagged_articles_coll.update_one({'_id':ObjectId(a['_id'])}, {'$set':{'taggedText':a['text'], 'taggedEntities':a['entities']}})
        except KeyError:
            pass

In [23]:
def updateEntityData(entity):
    global articles,tagged_articles_coll
    articleIds = entity['articleIds']
    names = entity['aliases']
    
    for articleId in articleIds:
        article = tagged_articles_coll.find({'_id':ObjectId(articleId)})
        if article.count() == 0:
            print(articleId, 'article not found')
            continue
        article = article[0]
        text = article['taggedText']
        try:
            entities = article['taggedEntities']
        except KeyError:
            print(articleId, 'no entities present')
            continue
        
        for e in entities:
            if e['name'] in names:
                for inst in e['instances']:
                    old_offset = inst['offset']
                    try:
                        (text, shift) = updateText(entity['_id'], str(text), e['type'], inst['offset'], inst['length'])
                    except TypeError:
                        print(text)
                    entities = updateOffset(entities, old_offset, shift)
        #print(articleId)
        tagged_articles_coll.update_one(filter={'_id':ObjectId(articleId)}, update={'$set':{'taggedEntities':entities, 'taggedText':text}})

In [26]:
copyText()

In [28]:
for e in resolved_entities.find(no_cursor_timeout=True):
    try:
        updateEntityData(e)
    except Exception as e:
        print(e)
        pass

In [ ]:
with open('resolvedArticles.txt', 'w') as f:
    for a in taggedArticles:
        print(taggedArticles[a]['taggedText'], file=f)

In [ ]:
!wc -l resolvedArticles.txt

In [22]:
resolved_entities.find({'graphid':{'$exists':True}}).count()

268094